In [ ]:
import os
import pickle
import sys
import time

import numpy as np
from pprint import pprint
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
from lenstronomy.Util import data_util
from lenstronomy.SimulationAPI.ObservationConfig.Roman import Roman
from hydra import initialize, compose
from pyHalo.preset_models import CDM
from pyHalo.plotting_routines import plot_multiplane_convergence

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):  # ../../
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.utils import util

In [ ]:
lens = SampleStrongLens()

z_lens = round(lens.z_lens, 2)
z_source = round(lens.z_source, 2)
log_m_host = np.log10(lens.get_main_halo_mass())
r_tidal = 0.25

subhalo_cone = 11
los_normalization = 0.

In [ ]:
cdm_realization = CDM(z_lens, z_source, log_m_host=log_m_host, r_tidal=r_tidal,
                          cone_opening_angle_arcsec=subhalo_cone,
                          LOS_normalization=los_normalization)

print(f'Realization contains {len(cdm_realization.halos)} halos.')

In [ ]:
# pickle it
with open(os.path.join(pickle_dir, 'cdm_subhalos_for_sample'), 'ab') as results_file:
        pickle.dump(cdm_realization, results_file)

In [ ]:
# unpickle to reproduce plots
with open(os.path.join(pickle_dir, 'cdm_subhalos_for_sample'), 'rb') as results_file:
        cdm_realization = pickle.load(results_file)

In [ ]:
print(len(cdm_realization.halos))

In [ ]:
plot_multiplane_convergence(cdm_realization, 
                                  npix=60,
                             cone_opening_angle_arcsec=subhalo_cone, 
                             lens_model_list_macro=None, 
                             kwargs_lens_macro=None, 
                             redshift_list_macro=None,
                                 show_critical_curve=False,
                                 vmin_max=0.01)

In [ ]:
cdm_halo_masses = [halo.mass for halo in cdm_realization.halos]
N, m = np.histogram(cdm_halo_masses, bins=np.logspace(6, 10, 10))

plt.loglog(m[0:-1], N)
plt.xlabel('Subhalo mass ('+r'$M_{\odot}$)')
plt.ylabel('Number of subhalos')

plt.savefig(os.path.join(figure_dir, 'shmf.png'))
plt.show()

In [ ]:
print(cdm_realization.astropy_instance)